# Agenda:
    - Hardware
    - Abstraction
    - GIL
    - LEGB Rule / scoped-based
    - co-routines
    - decorators
    
    - Intro towards Parallel-programming(hardware & packages)
    - MultiThreading.
    - Multiprocessing.
    - File-handling and serializing

### Process-Segments:
     Main-Thread :
    - Code-segment
        - functions <instructions>
    - Data-Segment
        - Global/Builtins
    - Kernel-Stack (managed by OS)
        - context-switching
    - PCB(managed by OS)
        - State of the process
    - User-Stack
        - Symbol/Scope-tables
    - Heap / free-store
        - Data

        Child-Threads(LWP <Light-Weight-Process>): Each child-thread shares parent segments => Code, Data, Heap
            TLS (Thread-Local-Storage) :-
                - user-Stack
                    - Symbol/Scope-tables
                - Kernel-Stack (managed by OS)
                    - thread-specific-context-switching
                - reserved-registers (specific to thread)
                - TCB (Thread-Control-Block)


In [1]:
'''
# Python: 
    - line-by-line Interpreter
    - top-down
    - dynamic-language
    - scope-based (GIL) and follows-LEGB-Rules
'''

'''
locking machanism at interpreter-level => GIL

GIL = lock() # Global-builtin-lock

'''

def gfun1():
    # acquire GIL & lock-it
    print(' gun1111')
    # Release acquired GIL & un-lock-it

def gfun2():
    # acquire GIL & lock-it
    print(' gun2222')
    # Release acquired GIL & un-lock-it
    
def main():
    # acquire GIL & lock-it
    
    # unlock-GIL & make a call to gfun1
    gfun1()
    # lock-GIL & continue
    print("+- "*25)
    
    # unlock-GIL & make a call to gfun2
    gfun2()
    # lock-GIL & continue main-function...
    
    
    # release acquired GIL & un-lock-it
    
main()



 gun1111
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 gun2222


# LEGB Rule

In [2]:
# LEGB Rule

gi = 10

def Outer():
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    print("- "*15)
    def Inner():
        print(f" .. Outer..Inner: gi = {gi}, id(gi) = {id(gi)}")
        
    Inner()
    print("- "*15)
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    
print("+- "*25)
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")
Outer()
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")

print("+- "*25)
    

+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Main: gi = 10, id(gi) = 1952445065808
 .. Outer: gi = 10, id(gi) = 1952445065808
- - - - - - - - - - - - - - - 
 .. Outer..Inner: gi = 10, id(gi) = 1952445065808
- - - - - - - - - - - - - - - 
 .. Outer: gi = 10, id(gi) = 1952445065808
 Main: gi = 10, id(gi) = 1952445065808
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 


In [3]:
# LEGB Rule
# L(ocal) => E(nclosed) => G(lobal) => B(uiltins) => raise NameError()


In [5]:
# LEGB Rule

gi = 10

def Outer():
    gi = 15
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    print("- "*15)
    def Inner():
        print(f" .. Outer..Inner: gi = {gi}, id(gi) = {id(gi)}")
        
    Inner()
    print("- "*15)
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    # del(local-symbol-table)
    # return None
    
print("+- "*25)
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")
Outer()
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")

print("+- "*25)
print(" Expected: output: 10, 15, 15, 15, 10")

+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Main: gi = 10, id(gi) = 1952445065808
 .. Outer: gi = 15, id(gi) = 1952445065968
- - - - - - - - - - - - - - - 
 .. Outer..Inner: gi = 15, id(gi) = 1952445065968
- - - - - - - - - - - - - - - 
 .. Outer: gi = 15, id(gi) = 1952445065968
 Main: gi = 10, id(gi) = 1952445065808
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Expected: output: 10, 15, 15, 15, 10


In [6]:
# LEGB Rule

gi = 10

def Outer():
    gi = 15
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    print("- "*15)
    def Inner():
        gi = 20
        print(f" .. Outer..Inner: gi = {gi}, id(gi) = {id(gi)}")
        
    Inner()
    print("- "*15)
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    # del(local-symbol-table)
    # return None
    
print("+- "*25)
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")
Outer()
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")

print("+- "*25)
print(" Expected: output: 10, 15, 20, 15, 10")

+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Main: gi = 10, id(gi) = 1952445065808
 .. Outer: gi = 15, id(gi) = 1952445065968
- - - - - - - - - - - - - - - 
 .. Outer..Inner: gi = 20, id(gi) = 1952445066128
- - - - - - - - - - - - - - - 
 .. Outer: gi = 15, id(gi) = 1952445065968
 Main: gi = 10, id(gi) = 1952445065808
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Expected: output: 10, 15, 20, 15, 10


In [7]:
# LEGB Rule

gi = 10

def Outer():
    gi = 15
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    print("- "*15)
    def Inner():
        global gi
        gi = 20
        print(f" .. Outer..Inner: gi = {gi}, id(gi) = {id(gi)}")
        
    Inner()
    print("- "*15)
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    # del(local-symbol-table)
    # return None
    
print("+- "*25)
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")
Outer()
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")

print("+- "*25)
print(" Expected: output: 10, 15, 20, 15, 20")

+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Main: gi = 10, id(gi) = 1952445065808
 .. Outer: gi = 15, id(gi) = 1952445065968
- - - - - - - - - - - - - - - 
 .. Outer..Inner: gi = 20, id(gi) = 1952445066128
- - - - - - - - - - - - - - - 
 .. Outer: gi = 15, id(gi) = 1952445065968
 Main: gi = 20, id(gi) = 1952445066128
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Expected: output: 10, 15, 20, 15, 20


In [8]:
# LEGB Rule

gi = 10

def Outer():
    gi = 15
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    print("- "*15)
    def Inner():
        #global gi
        nonlocal gi
        gi = 20
        print(f" .. Outer..Inner: gi = {gi}, id(gi) = {id(gi)}")
        
    Inner()
    print("- "*15)
    print(f" .. Outer: gi = {gi}, id(gi) = {id(gi)}")
    # del(local-symbol-table)
    # return None
    
print("+- "*25)
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")
Outer()
print(f" Main: gi = {gi}, id(gi) = {id(gi)}")

print("+- "*25)
print(" Expected: output: 10, 15, 20, 20, 10")

+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Main: gi = 10, id(gi) = 1952445065808
 .. Outer: gi = 15, id(gi) = 1952445065968
- - - - - - - - - - - - - - - 
 .. Outer..Inner: gi = 20, id(gi) = 1952445066128
- - - - - - - - - - - - - - - 
 .. Outer: gi = 20, id(gi) = 1952445066128
 Main: gi = 10, id(gi) = 1952445065808
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 Expected: output: 10, 15, 20, 20, 10


In [9]:
import _thread

In [11]:
import threading

In [12]:
dir(threading)

['Barrier',
 'BoundedSemaphore',
 'BrokenBarrierError',
 'Condition',
 'Event',
 'ExceptHookArgs',
 'Lock',
 'RLock',
 'Semaphore',
 'TIMEOUT_MAX',
 'Thread',
 'ThreadError',
 'Timer',
 'WeakSet',
 '_CRLock',
 '_DummyThread',
 '_HAVE_THREAD_NATIVE_ID',
 '_MainThread',
 '_PyRLock',
 '_RLock',
 '_SHUTTING_DOWN',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_active',
 '_active_limbo_lock',
 '_after_fork',
 '_allocate_lock',
 '_count',
 '_counter',
 '_dangling',
 '_deque',
 '_enumerate',
 '_islice',
 '_limbo',
 '_main_thread',
 '_maintain_shutdown_locks',
 '_make_invoke_excepthook',
 '_newname',
 '_os',
 '_profile_hook',
 '_register_atexit',
 '_set_sentinel',
 '_shutdown',
 '_shutdown_locks',
 '_shutdown_locks_lock',
 '_start_new_thread',
 '_sys',
 '_threading_atexits',
 '_time',
 '_trace_hook',
 'activeCount',
 'active_count',
 'currentThread',
 'current_thread',
 'enumerate',
 'excepthook',
 'functools',
 

In [13]:
help(threading.Thread)

Help on class Thread in module threading:

class Thread(builtins.object)
 |  Thread(group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None)
 |  
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None)
 |      This constructor should always be called with keyword arguments. Arguments are:
 |      
 |      *group* should be None; reserved for future extension when a ThreadGroup
 |      class is implemented.
 |      
 |      *target* is the callable object to be invoked by the run()
 |      method. Defaults to None, meaning nothing is called.
 |      
 |      *name* is the thread name. By default, a unique name is constructed of
 |      t

In [15]:
help(threading.currentThread)

Help on function current_thread in module threading:

current_thread()
    Return the current Thread object, corresponding to the caller's thread of control.
    
    If the caller's thread of control was not created through the threading
    module, a dummy thread object with limited functionality is returned.



In [16]:
help(threading.current_thread)

Help on function current_thread in module threading:

current_thread()
    Return the current Thread object, corresponding to the caller's thread of control.
    
    If the caller's thread of control was not created through the threading
    module, a dummy thread object with limited functionality is returned.



In [14]:

### Child-thread-functions
def child1_fun1():
    print('def child1_fun1() invoked...')

### Main-thread-app ###
def main():
    print(" Main-thread - starts ")
    t1 = threading.Thread(target=child1_fun1 )
    print('t1 = ', t1)

    print(" Main-thread - ends ")
    
if __name__ == '__main__':
    main()

 Main-thread - starts 
t1 =  <Thread(Thread-8, initial)>
 Main-thread - ends 


In [21]:
import time

### Child-thread-functions
def child1_fun1():
    print('def child1_fun1() invoked... started', end=":"*5+"\n")
    time.sleep(2)
    print('def child1_fun1() invoked...ended', end=":"*5+"\n")

### Main-thread-app ###
def main():
    print(" Main-thread - starts ")
    t1 = threading.Thread(target=child1_fun1 )
    print('t1 = ', t1)

    t1.start()
    print("+- "*25)
    
    time.sleep(1)
    t1.join()
    print(" Main-thread - ends ")
    
if __name__ == '__main__':
    main()

 Main-thread - starts 
t1 =  <Thread(Thread-13, initial)>
def child1_fun1() invoked... started:::::
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
def child1_fun1() invoked...ended:::::
 Main-thread - ends 


In [22]:
import time

### Child-thread-functions
def child1_fun1():
    print('def child1_fun1() invoked... started')
    time.sleep(2)
    print('def child1_fun1() invoked...ended')

### Main-thread-app ###
def main():
    print(" Main-thread - starts ")
    t1 = threading.Thread(target=child1_fun1 )
    print('t1 = ', t1)

    t1.start()
    print("+- "*25)
    
    time.sleep(1)
    t1.join()
    print(" Main-thread - ends ")
    
if __name__ == '__main__':
    main()

 Main-thread - starts 
t1 =  <Thread(Thread-14, initial)>
def child1_fun1() invoked... started
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
def child1_fun1() invoked...ended
 Main-thread - ends 


In [23]:
import time

### Child-thread-functions
def child1_fun1():
    print('def child1_fun1() invoked... started')
    time.sleep(2)
    print('def child1_fun1() invoked...ended')

### Main-thread-app ###
def main():
    print(" Main-thread - starts ")
    t1 = threading.Thread(target=child1_fun1 )
    print('t1 = ', t1)

    t1.start()
    print("+- "*25)
    
    time.sleep(1)
    t1.join()
    print(" Main-thread - ends ")
    
if __name__ == '__main__':
    main()

 Main-thread - starts 
t1 =  <Thread(Thread-15, initial)>
def child1_fun1() invoked... started+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 

def child1_fun1() invoked...ended
 Main-thread - ends 
